Conclusion:  Using n grams makes predictive performance worse

Things to try to increase performance: 
-even more data
-separate models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis
-Look for transformations opps, etc
-model only using genre

In [10]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

TEMP_FOLDER = tempfile.gettempdir()

In [11]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [12]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [13]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [14]:
links[5] = text

In [15]:
links.drop_duplicates([5], inplace = True)


In [16]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [17]:
links.shape

(2903, 6)

In [19]:
text = links[5]
text.shape

(2903,)

In [20]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    ['when i first reviewed civilization vi back i...
2    [' ', 'there aren’t many games that make me fe...
3    ['overkill’s ', ' is an earnest attempt to del...
4    ["it's often with rose-colored glasses that i ...
Name: 5, dtype: object

In [21]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [22]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [23]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1    when i first reviewed civilization vi back in ...
2      there arent many games that make me feel as ...
3    overkills   is an earnest attempt to deliver a...
4    its often with rosecolored glasses that i look...
Name: 5, dtype: object

In [24]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [25]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [26]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [27]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#Stemming (removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach)
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#train['text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization (more effective option than stemming because 
# converts the word into its root word, rather than just stripping the suffices)
#*Preferred to stemming*#


text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    first reviewed civilization vi back 2016 award...
2    arent many game make feel cool warframe im bul...
3    overkill earnest attempt deliver cooperative a...
4    often rosecolored glass look back pokemons kan...
Name: 5, dtype: object

In [28]:
text = text.to_frame()
text['score'] = links[1]

In [29]:
text.head()

,5,score
0,development muscle behind ea dice massive shoo...,7.5
1,first reviewed civilization vi back 2016 award...,9.4
2,arent many game make feel cool warframe im bul...,8.6
3,overkill earnest attempt deliver cooperative a...,5.3
4,often rosecolored glass look back pokemons kan...,8.3


In [66]:
#train val split

trainval = text.sample(800)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval['score']
trainval.drop(['score'], axis = 1, inplace = True)
train.drop(['score'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [89]:
vectorizer = TfidfVectorizer(ngram_range = (1,2))
X = vectorizer.fit_transform(train[5])
xtest = vectorizer.transform(trainval[5])
X.shape

(2103, 1856384)

In [77]:
vectorizer2 = TfidfVectorizer()
X = vectorizer2.fit_transform(train[5])
xtest = vectorizer2.transform(trainval[5])

## XGBoost

In [344]:
#bayesian opt
yarray = y.values
dtrain = xgb.DMatrix(all_topics_numpy, label=yarray)
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7),
                                            'gamma': (0, 1),
                                            'colsample_bytree': (0.3, 0.9)})

xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')



|   iter    |  target   | colsam... |   gamma   | max_depth |
-------------------------------------------------------------
[19:39:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
[19:39:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[19:39:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned

[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 6 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 2 pruned nodes, max_depth=4
[19:39:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 6 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[19:39:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:39:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_

[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 4 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 4 pruned nodes, max_depth=4
[19:40:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=4
[19:40:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:40:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_

[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 4 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=4
[19:40:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 6 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 2 pruned nodes, max_depth=4
[19:40:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
[19:40:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 4 pruned nodes, max_

[19:40:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 14 pruned nodes, max_depth=4
[19:40:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 2 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 8 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 4 pruned nodes, max_depth=4
[19:40:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_depth=4
|  1        | -1.108    |  0.5575   |  0.5564   |  4.346    |
[19:40:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:40:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:40:50] src/tree/updater_prune.cc:74: 

...

[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 12 pruned nodes, max_depth=4
[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[19:47:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 14 pruned nodes, max_depth=4
[19:47:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 6 pruned nodes, max_depth=4
[19:47:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 16 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 8 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 12 pruned nodes, max_depth=4
[19:47:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 4 pruned nodes, max_depth=4
[19:47:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 2 pruned nodes, 

[19:48:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

[19:48:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:48:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_depth=3
[19:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_

[19:49:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 2 pruned nodes, max_depth=3
[19:49:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 2 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:49:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 2 pruned nodes, max_depth=3
[19:49:26] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 4 pruned nodes, max_depth=3
[19:49:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 2 pruned nodes, max_d

In [357]:
param = xgb_bo.max
param

{'target': -1.1006746666666667,
 'params': {'colsample_bytree': 0.8976729774092942,
  'gamma': 0.9971962663965198,
  'max_depth': 4.321612175930944}}

In [340]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(all_topics_numpy,yarray)

predictions2 = xgbreg.predict(all_topics_numpytest)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.0728785941278771

## Lasso Regression

In [87]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

NameError: name 'all_topics_numpy' is not defined

In [88]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(X,yarray)

predictions3 = lasso.predict(xtest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


0.989588930540263

## OLS

In [90]:
yarray = y.values
lm = LinearRegression()
lm.fit(X,yarray)
predictions = lm.predict(xtest)
predictionsdf = pd.DataFrame(predictions)

In [91]:
predictionsdf.describe()

,0
count,800.000000
mean,7.397698
std,0.803636
min,4.531465
25%,6.895886
50%,7.467546
75%,7.930033
max,9.956699


In [92]:
valy = valy.reset_index()
valy

,index,score
0,0,8.4
1,1,8.0
2,2,6.5
3,3,9.3
4,4,7.1
5,5,3.0
6,6,7.6
7,7,7.0
8,8,8.5
9,9,7.3


In [93]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score
0,8.4
1,8.0
2,6.5
3,9.3
4,7.1
5,3.0
6,7.6
7,7.0
8,8.5
9,7.3


In [94]:
predictionsdf['true'] = valy
predictionsdf

,0,true
0,8.120015,8.4
1,7.270358,8.0
2,6.867380,6.5
3,8.659456,9.3
4,7.403476,7.1
5,6.172848,3.0
6,7.573723,7.6
7,7.153382,7.0
8,8.490235,8.5
9,8.020505,7.3


In [95]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

,0,true,diff
0,8.120015,8.4,0.279985
1,7.270358,8.0,0.729642
2,6.867380,6.5,-0.367380
3,8.659456,9.3,0.640544
4,7.403476,7.1,-0.303476
5,6.172848,3.0,-3.172848
6,7.573723,7.6,0.026277
7,7.153382,7.0,-0.153382
8,8.490235,8.5,0.009765
9,8.020505,7.3,-0.720505


In [96]:
#let's also plot histogram of scores




In [97]:
#residual plot here




In [98]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.0208139970520738

In [55]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

1.513717378488377